In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/hate/models

/content/drive/MyDrive/hate/models


In [3]:
! pip install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.1 MB/s eta 0:00:00


# BERT

In [ ]:
label_to_class= "offensive.language"  # enter the label to be classified
version = "E" # enter the version

In [4]:
import torch.nn as nn
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import time
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
     

In [5]:
# Define pretrained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
csv = pd.read_csv('../data/ourdata/version'+version+'_train.csv',header=0)

# create a new csv df
csv_new = pd.DataFrame(csv, columns=[label_to_class, "tweet_hashed"])
# drop all rows that have any NaN values
csv_new_clean = csv_new.dropna(axis=0,how="any")

In [ ]:
# calculate max length of the tweets
X= list(csv_new_clean["tweet_hashed"])
max_length = 0
for x in X:
    ids = tokenizer.encode(x)
    max_length = max(len(ids),max_length)

torch.save(max_length, "bert.max_length."+label_to_class+version)

In [ ]:
y = list(csv_new_clean[label_to_class])

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=max_length)
X_dev_tokenized = tokenizer(X_dev, padding=True, truncation=True, max_length=max_length)


In [6]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).long()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
dev_dataset = Dataset(X_dev_tokenized, y_dev)

In [ ]:
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    #labels= np.argmax(labels, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average = "weighted")
    precision = precision_score(y_true=labels, y_pred=pred, average = "weighted")
    f1 = f1_score(y_true=labels, y_pred=pred, average = "weighted")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer

args = TrainingArguments(
    output_dir="offensiveE",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    seed=42,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Train pre-trained model
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7840
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14700
  Number of trainable parameters = 109483778


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.531100,0.509114,0.808163,0.814866,0.808163,0.803414
1000,0.439400,0.442445,0.839286,0.844526,0.839286,0.836258
1500,0.386900,0.419578,0.848980,0.849127,0.848980,0.847995
2000,0.350600,0.470070,0.853571,0.854499,0.853571,0.852281
2500,0.350800,0.538883,0.847449,0.847207,0.847449,0.847290
3000,0.379600,0.434150,0.852041,0.851695,0.852041,0.851636


***** Running Evaluation *****
  Num examples = 1960
  Batch size = 8
Saving model checkpoint to offensiveE/checkpoint-500
Configuration saved in offensiveE/checkpoint-500/config.json
Model weights saved in offensiveE/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1960
  Batch size = 8
Saving model checkpoint to offensiveE/checkpoint-1000
Configuration saved in offensiveE/checkpoint-1000/config.json
Model weights saved in offensiveE/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1960
  Batch size = 8
Saving model checkpoint to offensiveE/checkpoint-1500
Configuration saved in offensiveE/checkpoint-1500/config.json
Model weights saved in offensiveE/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1960
  Batch size = 8
Saving model checkpoint to offensiveE/checkpoint-2000
Configuration saved in offensiveE/checkpoint-2000/config.json
Model weights saved in offensiveE/checkpoint-2000/pytor

# test the models

In [11]:

test_data_version = "E"

# init test file
test_csv = pd.read_csv('../data/ourdata/version'+test_data_version+'_test.csv', header=0)

# create a new csv df with all the original columns
test_csv_new = pd.DataFrame(test_csv, columns=["id", "version","batch.tweet","tweet.id", "tweet_hashed", "hate.speech", "offensive.language"])


In [12]:
for label_to_test in ["hate.speech", "offensive.language"]:
  for version_to_test in ["A","B","C","D","E"]:

    # preprocess the test data
    max_length = torch.load("bert.max_length."+label_to_test+version_to_test, map_location=DEVICE)

    X_test = tokenizer(list(test_csv_new["tweet_hashed"]), padding=True, truncation=True, max_length=max_length)

    test_dataset = Dataset(X_test)

    # Load trained model
    model_path = "bert.model."+label_to_test+version_to_test
    trained_model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

    # Define tester
    args = TrainingArguments(
        output_dir="tester",
        dataloader_pin_memory=False,
    )
    tester = Trainer(model=trained_model, args=args)

    # Make prediction
    raw_pred, _, _ = tester.predict(test_dataset)
    

    # Preprocess raw predictions
    y_pred = np.argmax(raw_pred, axis=1)

    if label_to_test == "hate.speech":
      if version_to_test == "A":
        column = 7
      elif version_to_test == "B":
        column = 8
      elif version_to_test == "C":
        column = 9
      elif version_to_test == "D":
        column = 10
      elif version_to_test == "E":
        column = 11
      else:
        raise KeyError
    elif label_to_test == "offensive.language":
      if version_to_test == "A":
        column = 12
      elif version_to_test == "B":
        column = 13
      elif version_to_test == "C":
        column = 14
      elif version_to_test == "D":
        column = 15
      elif version_to_test == "E":
        column = 16
      else:
        raise KeyError
    else:
      raise KeyError

    test_csv_new.insert(column,label_to_test+"_preds_"+version_to_test,y_pred)

test_csv_new

loading configuration file bert.model.hate.speechA/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechA/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechB/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechB/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechC/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechC/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechD/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechD/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

loading configuration file bert.model.hate.speechE/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.hate.speechE/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

...

loading configuration file bert.model.offensive.languageB/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageB/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

loading configuration file bert.model.offensive.languageC/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageC/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

loading configuration file bert.model.offensive.languageD/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageD/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

loading configuration file bert.model.offensive.languageE/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file bert.model.offensive.languageE/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of 

,id,version,batch.tweet,tweet.id,tweet_hashed,hate.speech,offensive.language,hate.speech_preds_A,hate.speech_preds_B,hate.speech_preds_C,hate.speech_preds_D,hate.speech_preds_E,offensive.language_preds_A,offensive.language_preds_B,offensive.language_preds_C,offensive.language_preds_D,offensive.language_preds_E
0,125,E,R1,1,@###### bro that hoe live,1.0,0,0,0,0,0,0,1,1,1,1,1
1,173,E,R1,1,@###### bro that hoe live,0.0,1,0,0,0,0,0,1,1,1,1,1
2,219,E,R1,1,@###### bro that hoe live,0.0,1,0,0,0,0,0,1,1,1,1,1
3,223,E,R1,1,@###### bro that hoe live,0.0,1,0,0,0,0,0,1,1,1,1,1
4,228,E,R1,1,@###### bro that hoe live,0.0,1,0,0,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,996,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,1,0,1,1,1,1,0,0,0,0,1
3331,1073,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,0,0,1,1,1,1,0,0,0,0,1
3332,1472,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,0,0,1,1,1,1,0,0,0,0,1
3333,1481,E,R49,2999,RT @###### My favorite episode of Friends is t...,0.0,0,0,1,1,1,1,0,0,0,0,1


In [13]:
test_csv_new.to_csv("../data/preds/bert_test"+test_data_version+".csv")